In [47]:
import datetime
import random
import time

import baostock as bs
import pandas as pd

from tqdm import tqdm

In [53]:
start_date = '1999-01-01'
end_date = '2021-07-02'

file_path = 'data/peTTM.csv.zip'

def print_error(rs):
    if rs.error_code != '0':
        print('respond error_code:'+rs.error_code)
        print('respond  error_msg:'+rs.error_msg)

        
bs.login()
stock_rs = bs.query_all_stock(end_date)
print_error(stock_rs)
stock_df = stock_rs.get_data()
stock_df = stock_df[stock_df['code_name'].str.contains('指数') == False]

data_df = pd.read_csv(file_path)
existing_code = data_df['code'].unique()

for i, code in enumerate(tqdm(stock_df["code"])):
    # skip already processed stock
    if code in existing_code:
        continue 
    
    # only process stock
    rs = bs.query_stock_basic(code=code)
    print_error(rs)    
    if rs.get_data()['type'].iloc[0] != '1':
        continue      
        
    k_rs = bs.query_history_k_data_plus(code, "date,code,peTTM", start_date, end_date)
    print_error(k_rs)
    
    data_df = data_df.append(k_rs.get_data())    
    if i % 100 == 0:
        data_df.to_csv(file_path, index=False)    
    time.sleep(random.random())

bs.logout()

login success!


100%|██████████| 4476/4476 [58:17<00:00,  1.28it/s]  


logout success!


In [53]:
rs = bs.query_history_k_data_plus("sh.600320", "date,code,peTTM", date, date)
df = df.append(rs.get_data(), ignore_index=True)

In [36]:
stock_df

""


In [57]:
rs = bs.query_trade_dates(start_date="1999-01-01", end_date="2021-07-04")

In [58]:
rs.error_code

'0'

In [80]:
df = bs.query_all_stock(date).get_data()

In [63]:
date.today().isoformat()

'2021-07-04'

In [61]:
from datetime import date

In [88]:
df[df['code_name'].str.contains('指数') == False]

,code,tradeStatus,code_name
8,sh.600001,1,邯郸钢铁
9,sh.600002,1,齐鲁石化
10,sh.600009,1,上海机场
11,sh.600051,1,宁波联合
12,sh.600052,1,浙江广厦
...,...,...,...
832,sz.000895,1,双汇发展
833,sz.000898,1,鞍钢股份
834,sz.000899,1,赣能股份
835,sz.001696,1,宗申动力


In [43]:
rs = bs.query_stock_basic(code=code)

In [92]:
bs.login()

rs = bs.query_trade_dates(start_date=start_date, end_date=end_date)
print_error(rs)
df = rs.get_data()
trading_day = df[df['is_trading_day'] == '1']['calendar_date']

login success!


In [95]:
trading_day[0::11]

3       1999-01-04
18      1999-01-19
33      1999-02-03
67      1999-03-09
82      1999-03-24
           ...    
8148    2021-04-23
8168    2021-05-13
8183    2021-05-28
8201    2021-06-15
8216    2021-06-30
Name: calendar_date, Length: 496, dtype: object

In [44]:
rs.get_data()['type'].iloc[0] != '1'

'1'

,date,code,peTTM
0,1999-01-04,sh.600001,12.507194
1,1999-01-04,sh.600002,15.482130
2,1999-01-04,sh.600009,37.231581
3,1999-01-04,sh.600051,31.467721
4,1999-01-04,sh.600052,27.401695
...,...,...,...
253865,2008-06-04,sz.399007,NaN
253866,2008-06-04,sz.399319,NaN
253867,2008-06-04,sz.399361,NaN
253868,2008-06-04,sz.399364,NaN


In [30]:
code = 'sh.600036'
date = '2020-06-01'
bs.login()
rs = bs.query_history_k_data_plus(code, "date,code,close,peTTM", date, date, adjustflag="2")
print(rs.get_data())
bs.logout

login success!
         date       code          close     peTTM
0  2020-06-01  sh.600036  33.7730322400  9.202857


<function baostock.login.loginout.logout(user_id='anonymous')>

In [33]:
k_rs = bs.query_history_k_data_plus(code, "date,code,peTTM", '1999-01-01', '2021-07-04')

In [32]:
bs.login()

login success!


In [34]:
k_rs.get_data()

,date,code,peTTM
0,2002-04-09,sh.600036,42.508441
1,2002-04-10,sh.600036,42.269182
2,2002-04-11,sh.600036,41.950169
3,2002-04-12,sh.600036,42.149552
4,2002-04-15,sh.600036,41.431773
...,...,...,...
4668,2021-06-28,sh.600036,13.583088
4669,2021-06-29,sh.600036,13.359600
4670,2021-06-30,sh.600036,13.456445
4671,2021-07-01,sh.600036,13.905903
